In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
from mars.movement import *
import numpy as np

In [3]:
from mars.action import Action, ActionType
import mars.serializer as ms
from collections import namedtuple

In [4]:
from pymongo import MongoClient
from bson import ObjectId

In [5]:
work_point_2 = ['asna2392-3-04.251','asna2392-3-04.261','asna2392-3-04.252','asna2392-3-04.260','asna2392-3-04.253','asna2392-3-04.259','asna2392-3-04.254','asna2392-3-04.258','asna2392-3-04.255','asna2392-3-04.257','asna2392-3-04.256','asna2392-3-04.250','asna2392-3-04.249','asna2392-3-04.248','asna2392-3-04.247','asna2392-3-04.246','asna2392-3-04.245','asna2392-3-04.244','asna2392-3-04.243','asna2392-3-04.242','asna2392-3-04.241','asna2392-3-04.240']

work_point_5 = ['asna2392-3-04.382','asna2392-3-04.383','asna2392-3-04.381','asna2392-3-04.384','asna2392-3-04.380','asna2392-3-04.385','asna2392-3-04.379','asna2392-3-04.386','asna2392-3-04.378','asna2392-3-04.387','asna2392-3-04.377','asna2392-3-04.376','asna2392-3-04.375','asna2392-3-04.374','asna2392-3-04.373','asna2392-3-04.372','asna2392-3-04.371','asna2392-3-04.370','asna2392-3-04.369','asna2392-3-04.368','asna2392-3-04.366','asna2392-3-04.367']

work_point_6 = ['asna2392-3-04.425','asna2392-3-04.426','asna2392-3-04.427','asna2392-3-04.428','asna2392-3-04.429','asna2392-3-04.430','asna2392-3-04.431','asna2392-3-04.432','asna2392-3-04.434','asna2392-3-04.433','asna2392-3-04.435','asna2392-3-04.419','asna2392-3-04.424','asna2392-3-04.418','asna2392-3-04.423','asna2392-3-04.417','asna2392-3-04.422','asna2392-3-04.416','asna2392-3-04.421','asna2392-3-04.415','asna2392-3-04.420','asna2392-3-04.414']

In [6]:
width_rail = 8
delta_station = -260.5
yrail_2 = -566.5
yrail_3 = -1077.5
yrail_4 = -1585.5
yrail_5 = -2096.5
yrail_6 = -2623.5

varObj = {
    'rail1':{
        'e1': 2651.469,
        'yUF1': 30,
        'yUF3': 2597.7
    },
    'rail2':{
        'e1': 2124.469,
        'yUF1': -497,
        'yUF3': 2070.7
    },
    'rail3':{
        'e1': 1613.469,
        'yUF1': -1008,
        'yUF3': 1559.7
    },
    'rail4':{
        'e1': 1105.469,
        'yUF1': -1516,
        'yUF3': 1051.7
    },
    'rail5':{
        'e1': 594.469,
        'yUF1': -2027,
        'yUF3': 540.7
    },
    'rail2':{
        'e1': 67.469,
        'yUF1': -2554,
        'yUF3': 13.7
    },
}


def rail_2_operation(origin_df):

    df = origin_df.copy()

    df['e1'] = yrail_2 + delta_station
    df.loc[df.position_type == 'CRT', 'y_j2'] = yrail_2 - (width_rail/2)
    df.loc[df.position_type == 'CRT', 'w_j4'] = -90.0
    df.loc[df.position_type == 'CRT', 'r_j6'] = 0.0

    return df

def rail_5_operation(origin_df):

    df = origin_df.copy()

    df.loc[df.position_type == 'JOINT', 'e1'] = yrail_5 - delta_station
    df.loc[df.position_type == 'JOINT', 'x_j1'] = origin_df.x_j1 *-1
    df.loc[df.position_type == 'JOINT', 'w_j4'] = origin_df.w_j4 *-1
    df.loc[df.position_type == 'JOINT', 'r_j6'] = origin_df.r_j6 *-1

    df.loc[df.position_type == 'CRT', 'e1'] = yrail_5 - delta_station +80
    df.loc[df.position_type == 'CRT', 'y_j2'] = yrail_5 - (width_rail/2)
    df.loc[df.position_type == 'CRT', 'w_j4'] = 90.0
    df.loc[df.position_type == 'CRT', 'r_j6'] = 0.0

    return df

def rail_6_operation(origin_df):

    df = origin_df.copy()

    df.loc[df.position_type == 'JOINT', 'e1'] = yrail_6 - delta_station
    df.loc[df.position_type == 'JOINT', 'x_j1'] = origin_df.x_j1 *-1
    df.loc[df.position_type == 'JOINT', 'w_j4'] = origin_df.w_j4 *-1
    df.loc[df.position_type == 'JOINT', 'r_j6'] = origin_df.r_j6 *-1

    df.loc[df.position_type == 'CRT', 'e1'] = yrail_6 - delta_station + 80
    df.loc[df.position_type == 'CRT', 'y_j2'] = yrail_6 - (width_rail/2)
    df.loc[df.position_type == 'CRT', 'w_j4'] = 90.0
    df.loc[df.position_type == 'CRT', 'r_j6'] = 0.0

    return df


In [34]:

def get_point_from_row(df_row_data):

    vector = np.array([
        df_row_data.x_j1,
        df_row_data.y_j2,
        df_row_data.z_j3,
        df_row_data.w_j4,
        df_row_data.p_j5,
        df_row_data.r_j6
    ])

    e1 = df_row_data.e1
    speed = df_row_data.speed
    cnt = df_row_data.cnt
    path = df_row_data.path
    pos_type = df_row_data.position_type

    if pos_type == "JOINT":
        position = PositionJoint(vector, e1)
    else:
        wrist = WristConfig.FLIP if df_row_data.wrist =='F' else WristConfig.NOFLIP
        forearm = ForeArmConfig.UP if df_row_data.forearm == 'U' else ForeArmConfig.DOWN
        arm = ArmConfig.TOWARD if df_row_data.arm =='T' else ArmConfig.BACKWARD

        config = Configuration(wrist, forearm, arm)
        position = PositionCrt(vector, e1, config)

    path = Path[df_row_data.path]

    return Point(cnt, speed, path, position)


def serialize_for_mongo(action, up_dependences_ids, down_dependences_ids):
    
    action_doc = ms.to_dict(action, database=True)
    del(action_doc['id'])

    if up_dependences_ids:
        action_doc['upstream_dependences'] = up_dependences_ids

    if down_dependences_ids:
        action_doc['downstream_dependences'] = down_dependences_ids

    return action_doc

def save_in_mongo(action, up_dependences_ids=None, down_dependences_ids=None):
    
    c = MongoClient()
    carrier = c.get_database('mars').get_collection('carrier')

    if type(action) == list:
        actions_docs = []
        for a in action:
            if type(a) == tuple:
                action_doc = serialize_for_mongo(a[0], up_dependences_ids, down_dependences_ids)
                action_doc.update(a[1])
            else:
                action_doc = serialize_for_mongo(a, up_dependences_ids, down_dependences_ids)
            
            actions_docs.append(action_doc)

        res = carrier.insert_many(actions_docs)

        if not res.acknowledged :
            print('Exception during insertion')
            return None
        else :
            print('insertion success, document id : ', str(res.inserted_ids))
            return res.inserted_ids
    
    else:
        if type(action) == tuple:
            action_doc = serialize_for_mongo(action[0], up_dependences_ids, down_dependences_ids)
            action_doc.update(action[1])
        else:
            action_doc = serialize_for_mongo(action, up_dependences_ids, down_dependences_ids)
        
        res = carrier.insert_one(action_doc)

        if not res.acknowledged :
            print('Exception during insertion')
            return None
        else :
            print('insertion success, document id : ', str(res.inserted_id))
            return res.inserted_id
    


def save_movements_from_df(master_df, generic_id, go_home_id, operation=None, description="", points_ids=None):

    if operation:
        print('apply operation to initial df')
        master_df = operation(master_df)

    # move station phase
    print('get station movement informations')
    station_df = master_df[master_df.mvt == "station"]
    points =[]
    for line in station_df.iterrows():
        points.append(get_point_from_row(line[1]))

    movement = Movement(1, 1, points)
    print('generate station movement action')
    action = (Action(0, ActionType['MOVE']['STATION']['WORK'],
        movement,
        'station movement for {}'.format(description)), {'target': {'type':'rail', 'id':generic_id}})

    print('save station movement action in database')
    station_db_id = save_in_mongo(action, up_dependences_ids=[go_home_id])
    if not station_db_id:
        raise Exception('Exception during station insertion')

    # update station df with database id
    station_df['database_id'] = str(station_db_id)


    # clearance phase
    print('get clearance movement informations')
    clearance_df = master_df[master_df.mvt == "clearance"]
    points = []

    for line in clearance_df.iterrows():
        points.append(get_point_from_row(line[1]))
    
    movement = Movement(1, 1, points)
    print('generate CLEARANCE movement action')
    action = (Action(0, ActionType['MOVE']['ARM']['CLEARANCE'],
        movement,
        'clearance movement for {}'.format(description)), {'target': {'type':'rail', 'id':generic_id}})

    print('save clearance action in database')
    clearance_db_id = save_in_mongo(action)
    if not clearance_db_id:
        raise Exception('Exception during clearance insertion')
    # update clearance_df with database id
    clearance_df['database_id'] = str(clearance_db_id)

    # approach phase
    print('get approach movement informations')
    approach_df = master_df[master_df.mvt == "approach"]
    points = []

    for line in approach_df.iterrows():
        points.append(get_point_from_row(line[1]))
    
    movement = Movement(1, 1, points)
    print('generate approach movement action')
    action = (Action(0, ActionType['MOVE']['ARM']['APPROACH'],
        movement,
        'approach movement for {}'.format(description)), {'target': {'type':'rail', 'id':generic_id}})

    print('save approach action in database')
    approach_db_id = save_in_mongo(action, up_dependences_ids = [station_db_id], down_dependences_ids = [clearance_db_id])
    if not approach_db_id:
        raise Exception('Exception during approach insertion')
    # update clearance_df with database id
    approach_df['database_id'] = str(approach_db_id)


    # work phase
    work_df = master_df[master_df.mvt == "work"]
    work_df.reset_index(inplace=True, drop=True)
    
    if points_ids :
        pid_serie = pd.Series(points_ids)
        work_df['id'] = pid_serie

    actions=[]
    for line in work_df.iterrows():
        point = get_point_from_row(line[1])
        movement = Movement(1,1, [point])
        action_description  = "work movement for {0} on {1}".format(line[1].id, description)
        action = Action(0, ActionType['MOVE']['ARM']['WORK'], movement, action_description)
        # j'ajoute une tuple dans mon tableau pour ajouter des info suplementaire à l'action
        actions.append((action, {'target': {'type':'fastener', 'id': line[1].id}}))

    work_db_ids = save_in_mongo(actions, up_dependences_ids=[approach_db_id]) 
    if not work_db_ids:
        raise Exception('Exception during approach works insertions')

    work_df['database_id'] = pd.Series(work_db_ids)

    global_df = pd.concat([station_df, clearance_df, approach_df, work_df], ignore_index=True)

    return global_df


    

In [43]:
# save home
home_df=pd.read_csv('./data/home.csv')
points = []
for line in home_df.iterrows():
  points.append(get_point_from_row(line[1]))

movement = Movement(1, 1, points)
print('generate go home movement action')
go_home_action = (Action(0, ActionType['MOVE']['STATION']['HOME'],
movement,
'go home movement'), {'target': {'type':'reference', 'id':'home'}})

print('generate return home movement action')
return_home_action = (Action(0, ActionType['MOVE']['STATION']['HOME'],
movement,
'return home movement'), {'target': {'type':'reference', 'id':'home'}})


print('return home movement action in database')
return_home_db_id = save_in_mongo(return_home_action)
if not return_home_db_id:
  raise Exception('Exception during station insertion')

print('save go home movement action in database')
go_home_db_id = save_in_mongo(go_home_action, down_dependences_ids=[return_home_db_id])
if not go_home_db_id:
  raise Exception('Exception during station insertion')

# update station df with database id
home_df['database_id'] = str(go_home_db_id)

generate go home movement action
generate return home movement action
return home movement action in database
insertion success, document id :  615da5e93d7104c1f1d3f958
save go home movement action in database
insertion success, document id :  615da5e93d7104c1f1d3f95a


In [44]:
r1_df=pd.read_csv('./data/rail1.csv')
r3_df=pd.read_csv('./data/rail3.csv')
r4_df=pd.read_csv('./data/rail4.csv')

In [45]:
r1_df_ = save_movements_from_df(r1_df, 1, go_home_db_id, operation=None, description='rail 1', points_ids=None)
r1_df.to_csv('./data/rail1.csv', index=False)

get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5ea3d7104c1f1d3f95c
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5ea3d7104c1f1d3f95e
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5eb3d7104c1f1d3f960
insertion success, document id :  [ObjectId('615da5eb3d7104c1f1d3f962'), ObjectId('615da5eb3d7104c1f1d3f963'), ObjectId('615da5eb3d7104c1f1d3f964'), ObjectId('615da5eb3d7104c1f1d3f965'), ObjectId('615da5eb3d7104c1f1d3f966'), ObjectId('615da5eb3d7104c1f1d3f967'), ObjectId('615da5eb3d7104c1f1d3f968'), ObjectId('615da5eb3d7104c1f1d3f969'), ObjectId('615da5eb3d7104c1f1d3f96a'), ObjectId('615da5eb3d7104c1f1d3f96b'), ObjectId('615da5eb3d7104c1f1d3f96c'), ObjectId('615da5eb3d7104c1f1d3f96d'), ObjectId('615da5e

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [46]:
r2_df = save_movements_from_df(r1_df, 2, go_home_db_id, operation = rail_2_operation, description="rail 2", points_ids=work_point_2)
r2_df.to_csv('./data/rail2.csv', index=False)

apply operation to initial df
get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5eb3d7104c1f1d3f979
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5eb3d7104c1f1d3f97b
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5eb3d7104c1f1d3f97d
insertion success, document id :  [ObjectId('615da5eb3d7104c1f1d3f97f'), ObjectId('615da5eb3d7104c1f1d3f980'), ObjectId('615da5eb3d7104c1f1d3f981'), ObjectId('615da5eb3d7104c1f1d3f982'), ObjectId('615da5eb3d7104c1f1d3f983'), ObjectId('615da5eb3d7104c1f1d3f984'), ObjectId('615da5eb3d7104c1f1d3f985'), ObjectId('615da5eb3d7104c1f1d3f986'), ObjectId('615da5eb3d7104c1f1d3f987'), ObjectId('615da5eb3d7104c1f1d3f988'), ObjectId('615da5eb3d7104c1f1d3f989'), ObjectId('615da5eb3d7104c

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [47]:
r5_df = save_movements_from_df(r1_df, 5, go_home_db_id, operation = rail_5_operation, description="rail 5", points_ids=work_point_5)
r5_df.to_csv('./data/rail5.csv', index=False)

apply operation to initial df
get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5ec3d7104c1f1d3f996
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5ec3d7104c1f1d3f998
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5ec3d7104c1f1d3f99a
insertion success, document id :  [ObjectId('615da5ec3d7104c1f1d3f99c'), ObjectId('615da5ec3d7104c1f1d3f99d'), ObjectId('615da5ec3d7104c1f1d3f99e'), ObjectId('615da5ec3d7104c1f1d3f99f'), ObjectId('615da5ec3d7104c1f1d3f9a0'), ObjectId('615da5ec3d7104c1f1d3f9a1'), ObjectId('615da5ec3d7104c1f1d3f9a2'), ObjectId('615da5ec3d7104c1f1d3f9a3'), ObjectId('615da5ec3d7104c1f1d3f9a4'), ObjectId('615da5ec3d7104c1f1d3f9a5'), ObjectId('615da5ec3d7104c1f1d3f9a6'), ObjectId('615da5ec3d7104c

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [48]:
r6_df = save_movements_from_df(r1_df, 6, go_home_db_id, operation = rail_6_operation, description="rail 6", points_ids=work_point_6)
r6_df.to_csv('./data/rail6.csv', index=False)

apply operation to initial df
get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5ec3d7104c1f1d3f9b3
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5ec3d7104c1f1d3f9b5
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5ec3d7104c1f1d3f9b7
insertion success, document id :  [ObjectId('615da5ec3d7104c1f1d3f9b9'), ObjectId('615da5ec3d7104c1f1d3f9ba'), ObjectId('615da5ec3d7104c1f1d3f9bb'), ObjectId('615da5ec3d7104c1f1d3f9bc'), ObjectId('615da5ec3d7104c1f1d3f9bd'), ObjectId('615da5ec3d7104c1f1d3f9be'), ObjectId('615da5ec3d7104c1f1d3f9bf'), ObjectId('615da5ec3d7104c1f1d3f9c0'), ObjectId('615da5ec3d7104c1f1d3f9c1'), ObjectId('615da5ec3d7104c1f1d3f9c2'), ObjectId('615da5ec3d7104c1f1d3f9c3'), ObjectId('615da5ec3d7104c

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [49]:
r3_df_ = save_movements_from_df(r3_df, 3, go_home_db_id, operation=None, description='rail 3', points_ids=None)
r3_df.to_csv('./data/rail3.csv', index=False)

get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9d0
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9d2
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9d4
insertion success, document id :  [ObjectId('615da5ed3d7104c1f1d3f9d6'), ObjectId('615da5ed3d7104c1f1d3f9d7'), ObjectId('615da5ed3d7104c1f1d3f9d8'), ObjectId('615da5ed3d7104c1f1d3f9d9'), ObjectId('615da5ed3d7104c1f1d3f9da'), ObjectId('615da5ed3d7104c1f1d3f9db'), ObjectId('615da5ed3d7104c1f1d3f9dc'), ObjectId('615da5ed3d7104c1f1d3f9dd'), ObjectId('615da5ed3d7104c1f1d3f9de'), ObjectId('615da5ed3d7104c1f1d3f9df'), ObjectId('615da5ed3d7104c1f1d3f9e0'), ObjectId('615da5ed3d7104c1f1d3f9e1'), ObjectId('615da5e

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [50]:
r4_df_ = save_movements_from_df(r4_df, 4, go_home_db_id, operation=None, description='rail 4', points_ids=None)
r1_df.to_csv('./data/rail4.csv', index=False)

get station movement informations
generate station movement action
save station movement action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9eb
get clearance movement informations
generate CLEARANCE movement action
save clearance action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9ed
get approach movement informations
generate approach movement action
save approach action in database
insertion success, document id :  615da5ed3d7104c1f1d3f9ef
insertion success, document id :  [ObjectId('615da5ed3d7104c1f1d3f9f1'), ObjectId('615da5ed3d7104c1f1d3f9f2'), ObjectId('615da5ed3d7104c1f1d3f9f3'), ObjectId('615da5ed3d7104c1f1d3f9f4'), ObjectId('615da5ed3d7104c1f1d3f9f5'), ObjectId('615da5ed3d7104c1f1d3f9f6'), ObjectId('615da5ed3d7104c1f1d3f9f7'), ObjectId('615da5ed3d7104c1f1d3f9f8'), ObjectId('615da5ed3d7104c1f1d3f9f9'), ObjectId('615da5ed3d7104c1f1d3f9fa'), ObjectId('615da5ed3d7104c1f1d3f9fb'), ObjectId('615da5ed3d7104c1f1d3f9fc'), ObjectId('615da5e

C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\mars\lib\site-packages\ipykernel_launcher.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio